# Customer & Sales Analytics: Segmentation, Forecasting, and Churn Prediction Using Python

---

## Business Objective
The objective of this project is to analyze historical e-commerce data in order to:
- Understand customer purchasing behavior
- Segment customers based on value and engagement
- Forecast future sales to support planning
- Predict customer churn and identify at-risk customers
- Provide actionable business recommendations to improve revenue retention
ten

---

## Key Business Questions
- Who are our most valuable customers?
- Which customers are likely to churn?
- How will sales behave in the near future?
- What actions can increase customer retention and revenue?


In [1]:
# import reuired libraries

# data manipulation
import pandas as pd
import numpy as np

# data visualization
import matplotlib.pyplot as plt
import seaborn as sns

# date & time handling
from datetime import timedelta  # date-based calculations

# machine learning libraries
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

# time series forecasting
from statsmodels.tsa.statespace.sarimax import SARIMAX

# ignore warnings for cleaner output
import warnings
warnings.filterwarnings("ignore")

# Step 1 - Data Ingestion

In [2]:
# load dataset files

customers = pd.read_csv("OneDrive/Documents/Customer Sales Analytics/olist_customers_dataset.csv")
orders = pd.read_csv("OneDrive/Documents/Customer Sales Analytics/olist_orders_dataset.csv")
order_items = pd.read_csv("OneDrive/Documents/Customer Sales Analytics/olist_order_items_dataset.csv")
payments = pd.read_csv("OneDrive/Documents/Customer Sales Analytics/olist_order_payments_dataset.csv")
reviews = pd.read_csv("OneDrive/Documents/Customer Sales Analytics/olist_order_reviews_dataset.csv")
products = pd.read_csv("OneDrive/Documents/Customer Sales Analytics/olist_products_dataset.csv")

In [3]:
# validate data loading

datasets = {
    "Customers": customers,
    "Orders": orders,
    "Order Items": order_items,
    "Payments": payments,
    "Reviews": reviews,
    "Products": products
}

for name, df in datasets.items():
    print(f"{name}: {df.shape}")

Customers: (99441, 5)
Orders: (99441, 8)
Order Items: (112650, 7)
Payments: (103886, 5)
Reviews: (99224, 7)
Products: (32951, 9)


# Step 2 - Data Cleaning & Master Table Creation

In [4]:
# check basic info for each dataset

datasets = {
    "Customers": customers,
    "Orders": orders,
    "Order Items": order_items,
    "Payments": payments,
    "Reviews": reviews,
    "Products": products
}

for name, df in datasets.items():
    print(f"\n{name} Dataset:")
    print(f" Shape: {df.shape}")
    print(f" Duplicates: {df.duplicated().sum()}")
    print(f" Missing Values: {df.isnull().sum().sum()}")
    print(f" Columns: {list(df.columns)}")


Customers Dataset:
 Shape: (99441, 5)
 Duplicates: 0
 Missing Values: 0
 Columns: ['customer_id', 'customer_unique_id', 'customer_zip_code_prefix', 'customer_city', 'customer_state']

Orders Dataset:
 Shape: (99441, 8)
 Duplicates: 0
 Missing Values: 4908
 Columns: ['order_id', 'customer_id', 'order_status', 'order_purchase_timestamp', 'order_approved_at', 'order_delivered_carrier_date', 'order_delivered_customer_date', 'order_estimated_delivery_date']

Order Items Dataset:
 Shape: (112650, 7)
 Duplicates: 0
 Missing Values: 0
 Columns: ['order_id', 'order_item_id', 'product_id', 'seller_id', 'shipping_limit_date', 'price', 'freight_value']

Payments Dataset:
 Shape: (103886, 5)
 Duplicates: 0
 Missing Values: 0
 Columns: ['order_id', 'payment_sequential', 'payment_type', 'payment_installments', 'payment_value']

Reviews Dataset:
 Shape: (99224, 7)
 Duplicates: 0
 Missing Values: 145903
 Columns: ['review_id', 'order_id', 'review_score', 'review_comment_title', 'review_comment_message

In [5]:
# check order status distribution
print("\nOrder Status Distribution")
print(orders['order_status'].value_counts())


Order Status Distribution
order_status
delivered      96478
shipped         1107
canceled         625
unavailable      609
invoiced         314
processing       301
created            5
approved           2
Name: count, dtype: int64


In [6]:
# Convert date columns to datetime
date_columns = [
    'order_purchase_timestamp',
    'order_approved_at',
    'order_delivered_carrier_date',
    'order_delivered_customer_date',
    'order_estimated_delivery_date'
]

for col in date_columns:
    orders[col] = pd.to_datetime(orders[col], errors='coerce')

In [7]:
orders[date_columns].dtypes

order_purchase_timestamp         datetime64[ns]
order_approved_at                datetime64[ns]
order_delivered_carrier_date     datetime64[ns]
order_delivered_customer_date    datetime64[ns]
order_estimated_delivery_date    datetime64[ns]
dtype: object

In [8]:
# check date range
print(f"\nDate Range:")
print(f"First Order: {orders['order_purchase_timestamp'].min()}")
print(f"Last Order: {orders['order_purchase_timestamp'].max()}")


Date Range:
First Order: 2016-09-04 21:15:19
Last Order: 2018-10-17 17:30:18


In [9]:
# keep only delivered orders
# for better sales analysis

orders_delivered = orders[orders["order_status"] == "delivered"].copy()

print("Original orders:", orders.shape)
print("Delivered orders:", orders_delivered.shape)

Original orders: (99441, 8)
Delivered orders: (96478, 8)


In [10]:
# Aggregation
# Order Items: Multiple items per order → aggregated (count, total price, freight)

# aggregate order items by order level

order_items_agg = (
    order_items
    .groupby("order_id", as_index=False)
    .agg(
        num_items=("order_item_id", "count"),
        num_unique_products=("product_id", "nunique"),
        total_item_price=("price", "sum"),
        total_freight_value=("freight_value", "sum")
    )
)

order_items_agg["order_revenue"] = (
    order_items_agg["total_item_price"] + order_items_agg["total_freight_value"]
)

order_items_agg.head()

,order_id,num_items,num_unique_products,total_item_price,total_freight_value,order_revenue
0,00010242fe8c5a6d1ba2dd792cb16214,1,1,58.90,13.29,72.19
1,00018f77f2f0320c557190d7a144bdd3,1,1,239.90,19.93,259.83
2,000229ec398224ef6ca0657da4fc703e,1,1,199.00,17.87,216.87
3,00024acbcdf0a6daa1e931b038114c75,1,1,12.99,12.79,25.78
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,1,199.90,18.14,218.04


In [11]:
# validation
# these two numbers should be samee

order_items_agg.shape
order_items_agg["order_id"].nunique()

98666

In [12]:
# aggregate payments to order level
# to capture total amount paid, payment complexity, and dominant payment method per order.

payments_agg = (
    payments
    .groupby("order_id", as_index=False)
    .agg(
        total_payment_value=("payment_value", "sum"),
        payment_count=("payment_sequential", "count"),
        max_installments=("payment_installments", "max"),
        primary_payment_type=(
            "payment_type",
            lambda x: x.mode().iloc[0] if not x.mode().empty else x.iloc[0]
        )
    )
)

payments_agg.head()

,order_id,total_payment_value,payment_count,max_installments,primary_payment_type
0,00010242fe8c5a6d1ba2dd792cb16214,72.19,1,2,credit_card
1,00018f77f2f0320c557190d7a144bdd3,259.83,1,3,credit_card
2,000229ec398224ef6ca0657da4fc703e,216.87,1,5,credit_card
3,00024acbcdf0a6daa1e931b038114c75,25.78,1,2,credit_card
4,00042b26cf59d7ce69dfabb4e55b4fd9,218.04,1,3,credit_card


In [13]:
# aggregate reviews to order level

reviews_agg = (
    reviews
    .groupby("order_id", as_index=False)
    .agg(
        review_score=("review_score", "mean"),
        review_count=("review_score", "count")
    )
)

reviews_agg.head()

,order_id,review_score,review_count
0,00010242fe8c5a6d1ba2dd792cb16214,5.0,1
1,00018f77f2f0320c557190d7a144bdd3,4.0,1
2,000229ec398224ef6ca0657da4fc703e,5.0,1
3,00024acbcdf0a6daa1e931b038114c75,4.0,1
4,00042b26cf59d7ce69dfabb4e55b4fd9,5.0,1


In [14]:
# master table creation
# create master table by merging all tables

# merging delivered order with customers
master_df = orders_delivered.merge(
    customers,
    on="customer_id",
    how="left"
)

print("Before merge (orders_delivered):", orders_delivered.shape)
print("After merging customers:", master_df.shape) 

Before merge (orders_delivered): (96478, 8)
After merging customers: (96478, 12)


In [15]:
# merge order items aggregation
master_df = master_df.merge(
    order_items_agg,
    on="order_id",
    how="left"
)

print("After merging order items:", master_df.shape) 

After merging order items: (96478, 17)


In [16]:
# merge payments aggregation
master_df = master_df.merge(
    payments_agg,
    on="order_id",
    how="left"
)

print("After merging payments:", master_df.shape)

After merging payments: (96478, 21)


In [17]:
# merge reviews aggregation
master_df = master_df.merge(
    reviews_agg,
    on="order_id",
    how="left"
)

print("After merging reviews:", master_df.shape)

After merging reviews: (96478, 23)


In [18]:
# Quick validation
# these two numbers must match
print("Total rows:", master_df.shape[0])
print("Unique orders:", master_df["order_id"].nunique())

Total rows: 96478
Unique orders: 96478


In [19]:
# master table dataset quality check

print("===== DUPLICATE CHECKS =====")

# check duplicate rows
print("Duplicate rows:", master_df.duplicated().sum())

# check duplicate columns
print("Duplicate order_ids:", master_df["order_id"].duplicated().sum())


print("===== MISSING VALUE CHECKS =====")

# missing value counts
missing_count = master_df.isnull().sum()

# missing value percentage
missing_percentage = (master_df.isnull().mean() * 100)

# combine into a single table
missing_summary = (
    pd.DataFrame({
        "missing_count": missing_count,
        "missing_percentage": missing_percentage
    })
    .query("missing_count > 0")
    .sort_values(by="missing_percentage", ascending=False)
)

missing_summary

===== DUPLICATE CHECKS =====
Duplicate rows: 0
Duplicate order_ids: 0
===== MISSING VALUE CHECKS =====


,missing_count,missing_percentage
review_score,646,0.669583
review_count,646,0.669583
order_approved_at,14,0.014511
order_delivered_customer_date,8,0.008292
order_delivered_carrier_date,2,0.002073
total_payment_value,1,0.001037
payment_count,1,0.001037
max_installments,1,0.001037
primary_payment_type,1,0.001037


In [20]:
# HANDLE MISSING VALUES + VERIFY

print("===== HANDLING MISSING VALUES =====")

# 1. Review-related features

# Flag to indicate whether a review exists
master_df["has_review"] = master_df["review_score"].notna().astype(int)

# 0 = no review given
master_df["review_score"] = master_df["review_score"].fillna(0)

# Optional smoothed version (useful for some models / EDA)
master_df["review_score_mean_imputed"] = (
    master_df["review_score"]
    .replace(0, np.nan)
    .fillna(master_df["review_score"].mean())
)

# Review count: missing means no review
master_df["review_count"] = master_df["review_count"].fillna(0)

print("Review features handled")

# 2. Payment-related features

payment_cols = [
    "total_payment_value",
    "payment_count",
    "max_installments"
]

for col in payment_cols:
    master_df[col] = master_df[col].fillna(0)

# Categorical fallback
master_df["primary_payment_type"] = master_df["primary_payment_type"].fillna("unknown")

print("Payment features handled")

# 3. Data type safety

numeric_cols = [
    "review_score",
    "review_score_mean_imputed",
    "review_count",
    "total_payment_value",
    "payment_count",
    "max_installments",
    "order_revenue"
]

master_df[numeric_cols] = master_df[numeric_cols].apply(
    pd.to_numeric, errors="coerce"
)

print("Numeric data types validated")

# VERIFICATION

print("\n===== VERIFICATION =====")

# Duplicate checks
print("Duplicate rows:", master_df.duplicated().sum())
print("Duplicate order_ids:", master_df["order_id"].duplicated().sum())

# Missing values summary
remaining_missing = master_df.isnull().sum().sort_values(ascending=False)
remaining_missing = remaining_missing[remaining_missing > 0]

print("\nRemaining missing values (if any):")
print(remaining_missing if not remaining_missing.empty else "No remaining missing values (except acceptable timestamps)")


===== HANDLING MISSING VALUES =====
Review features handled
Payment features handled
Numeric data types validated

===== VERIFICATION =====
Duplicate rows: 0
Duplicate order_ids: 0

Remaining missing values (if any):
order_approved_at                14
order_delivered_customer_date     8
order_delivered_carrier_date      2
dtype: int64


# Step 3 - Feature Engineering

These features will be crucial for:

- RFM Analysis (Recency, Frequency, Monetary calculation)
- Sales Forecasting (time trends)
- Churn Prediction (customer behavior patterns)

In [23]:
# Feature Engineering 

# 1. Delivery & Logistics related features

# calculate how long delivery took (in days)
master_df["delivery_time_days"] = (
    master_df["order_delivered_customer_date"] -
    master_df["order_purchase_timestamp"]
).dt.days

# calculate delivery delays (nagative = early, positive = late)
master_df["delivery_delay_days"] = (
    master_df["order_delivered_customer_date"] -
    master_df["order_estimated_delivery_date"]
).dt.days

# late delivery flag (1 = late, 0 = on-time or early)
master_df["is_late_delivery"] = (
    master_df["delivery_delay_days"] > 0
).astype(int)

print("Delivery performance features created")


# 2. core time features (forecasting + churn)

master_df["order_year"] = master_df["order_purchase_timestamp"].dt.year
master_df["order_month"] = master_df["order_purchase_timestamp"].dt.month
master_df["order_day_of_week"] = master_df["order_purchase_timestamp"].dt.dayofweek
master_df["is_weekend"] = master_df["order_day_of_week"].isin([5,6]).astype(int)

# time periods for aggregation
master_df["year_month"] = master_df["order_purchase_timestamp"].dt.to_period("M").astype(str)

print("Time-based features created")

# 3. order value & pricing features

# average item price
master_df["avg_item_price"] = (
    master_df["total_item_price"] / master_df["num_items"]
)

# freight as % of revenue
master_df["freight_percentage"] = (
    master_df["total_freight_value"] / master_df["order_revenue"]
).replace([np.inf, -np.inf], np.nan).fillna(0) * 100

print("Order value features created")

# 4. customer satisfaction signal

# base satisfaction = review score (already handles earlier)
# penalize late delivery mildly 
master_df["satisfaction_score"] = (
    master_df["review_score_mean_imputed"] *
    (1 - 0.15 * master_df["is_late_delivery"])
)

print("Satisfaction score created")

# 5. final data type safety

numeric_cols = [
    "delivery_time_days",
    "delivery_delay_days",
    "avg_item_price",
    "freight_percentage",
    "satisfaction_score"
]

master_df[numeric_cols] = master_df[numeric_cols].apply(
    pd.to_numeric, errors="coerce"
)

print("Numeric data types validated")

# 6. final validation

print("\n===== VALIDATION =====")
print("Duplicate rows:", master_df.duplicated().sum())
print("Duplicate order_ids:", master_df["order_id"].duplicated().sum())

new_features = [
    "delivery_time_days",
    "delivery_delay_days",
    "avg_item_price",
    "freight_percentage",
    "satisfaction_score"
]

missing_new = master_df[new_features].isnull().sum()
print("\nMissing values in engineered features:")
print(missing_new[missing_new > 0] if missing_new.sum() > 0 else "no missing values")

print("\n===== FEATURE ENGINEERING COMPLETE =====")
print(f"Total columns in master_df: {master_df.shape[1]}")

Delivery performance features created
Time-based features created
Order value features created
Satisfaction score created
Numeric data types validated

===== VALIDATION =====
Duplicate rows: 0
Duplicate order_ids: 0

Missing values in engineered features:
delivery_time_days     8
delivery_delay_days    8
dtype: int64

===== FEATURE ENGINEERING COMPLETE =====
Total columns in master_df: 37


In [24]:
# feature statistics

print("\n===== FEATURE STATISTICS =====")

# binary feature distributions
late_delivery_rate = master_df["is_late_delivery"].mean() * 100
weekend_order_rate = master_df["is_weekend"].mean() * 100

print(f"Late deliveries: {master_df['is_late_delivery'].sum():,} "
      f"({late_delivery_rate:.1f}%)")

print(f"Weekend orders: {master_df['is_weekend'].sum():,} "
      f"({weekend_order_rate:.1f}%)")

# continuous feature averages
print(f"Average delivery time: {master_df['delivery_time_days'].mean():.1f} days")
print(f"Average item price: ${master_df['avg_item_price'].mean():.2f}")
print(f"Average satisfaction score: {master_df['satisfaction_score'].mean():.2f}")


# sample of engineered features

print("\n===== SAMPLE OF ENGINEERED FEATURES =====")

sample_cols = [
    "order_id",
    "delivery_time_days",
    "is_late_delivery",
    "year_month",
    "avg_item_price",
    "satisfaction_score"
]

print(master_df[sample_cols].head(10))


# quick distribution check

print("\n===== QUICK DISTRIBUTION CHECKS =====")

print("\nDelivery time (days):")
print(master_df["delivery_time_days"].describe())

print("\nSatisfaction score:")
print(master_df["satisfaction_score"].describe())


===== FEATURE STATISTICS =====
Late deliveries: 6,534 (6.8%)
Weekend orders: 22,190 (23.0%)
Average delivery time: 12.1 days
Average item price: $125.23
Average satisfaction score: 4.13

===== SAMPLE OF ENGINEERED FEATURES =====
                           order_id  delivery_time_days  is_late_delivery  \
0  e481f51cbdc54678b7cc49136f2d6af7                 8.0                 0   
1  53cdb2fc8bc7dce0b6741e2150273451                13.0                 0   
2  47770eb9100c2d0c44946d9cf07ec65d                 9.0                 0   
3  949d5b44dbf5de918fe9c16f97b45f8a                13.0                 0   
4  ad21c59c0840e6cb83a9ceb5573f8159                 2.0                 0   
5  a4591c265e18cb1dcee52889e2d8acc3                16.0                 0   
6  6514b8ad8028c9f2cc2374ded245783f                 9.0                 0   
7  76c6e866289321a7c93b82b54852dc33                 9.0                 0   
8  e69bfb5eb88e0ed6a785585b27e16dbf                18.0                 0   


In [25]:
# in delivery time(days), the max time taken is shown to be 209 days
# which might be an outlier

# Check extreme delivery times
print(master_df[master_df['delivery_time_days'] > 100][
    ['order_id', 'delivery_time_days', 'is_late_delivery', 'review_score']
])

                               order_id  delivery_time_days  is_late_delivery  \
1578   a4efaffc506a395c9cea7402b078c1e5               110.0                 1   
2994   8b7fd198ad184563c231653673e75a7f               105.0                 1   
3116   4f39a94d6e474819d898d6df7d394996               143.0                 1   
4532   b31c7dea63bb08f8cdd1ec32514ccf0b               132.0                 1   
10092  3602a80b09d914236f74c733631f3b8b               106.0                 1   
...                                 ...                 ...               ...   
83931  ed8e9faf1b75f43ee027103957135663               173.0                 1   
86464  285ab9426d6982034523a855f55a885e               194.0                 1   
89466  29c3b79aace1b72a82b1232bf494e16f               133.0                 1   
92305  17cc6728043d53cc948551dfbf0a338b               142.0                 1   
95300  2e56f943f231f5fe108f43fb370b0ed6               104.0                 1   

       review_score  
1578 

In [26]:
# handling extreme values
# appling capping + flag
# capping - giving it maximum value (used percentile-based)

# data-driven cap
delivery_cap = master_df["delivery_time_days"].quantile(0.99)

# flag extreme delays
master_df["is_extreme_delay"] = (
    master_df["delivery_time_days"] > delivery_cap
).astype(int)

# cap delivery time
master_df["delivery_time_days_capped"] = (
    master_df["delivery_time_days"].clip(upper=delivery_cap)
)

# business validation
extreme = master_df[
    master_df['is_extreme_delay'] == 1
]['review_score'].mean()

normal = master_df[
    master_df['is_extreme_delay'] == 0
]['review_score'].mean()

print(f"Extreme delays: {master_df['is_extreme_delay'].sum()}")
print(f"Avg satisfaction (extreme): {extreme:.2f}")
print(f"Avg satisfaction (normal): {normal:.2f}")
print(f"Satisfaction drop: {normal - extreme:.2f} stars")

Extreme delays: 880
Avg satisfaction (extreme): 1.73
Avg satisfaction (normal): 4.15
Satisfaction drop: 2.42 stars


## Saving the Master Table

In [27]:
# Saving the master table for further analytics

print("\n===== Saving Master Table =====")

# final check on dimensions
print(f"Final shape: {master_df.shape}")
print(f"Total orders: {len(master_df):,}")
print(f"Total features: {master_df.shape[1]}")

# save to CSV file
master_df.to_csv(
    "OneDrive/Documents/Customer Sales Analytics/master_table_processed.csv",
    index=False
)
print("Master table saved to")


===== Saving Master Table =====
Final shape: (96478, 39)
Total orders: 96,478
Total features: 39
Master table saved to
